In [35]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time, json, os, csv

from scipy.stats import linregress
from citipy import citipy
from pprint import pprint

# Hi Markers, please use your onw API keys to run the script, thank you!
from api_keys import weather_api_key
from api_keys import g_key

In [36]:
# Define function to set data type
def set_type(data_type, pddf):
    for row in data_type:
        pddf[row[0]].astype(row[1])

integer = "int"

In [49]:
# Read data files
house_data_path = "data_source/Melbourne_housing_FULL.csv"
school_data_path = "data_source/dv309_schoollocations2021.csv"

house_data = pd.read_csv(house_data_path)
school_data = pd.read_csv(school_data_path, encoding = "cp1252")

# Set output file path
output_file = "data_source/house_school.csv"


In [38]:
house_data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,3/09/2016,2.5,3067.0,...,1.0,1.0,126.0,NaN,NaN,Yarra City Council,-37.8014,144.9958,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019.0
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,4/02/2016,2.5,3067.0,...,2.0,1.0,0.0,NaN,NaN,Yarra City Council,-37.8114,145.0116,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019.0


In [39]:
# cleaning house data
house_data_cleaned = house_data.dropna(subset = ["Price"])
house_data_cleaned = house_data_cleaned.drop(["Bedroom2"], axis = "columns")
house_data_cleaned = house_data_cleaned.fillna(0)

# Set up a list for data type
data_list = [["Postcode", integer],
             ["Bathroom", integer],
             ["Car", integer],
             ["Landsize", integer],
             ["BuildingArea", integer],
             ["Propertycount", integer],
             ["Rooms", integer]]

set_type(data_list, house_data_cleaned)

house_data_cleaned["Price"] = house_data_cleaned.apply(lambda x: "{:,.0f}".format(x["Price"]), axis = 1)

In [40]:
house_data_cleaned.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
1,Abbotsford,85 Turner St,2,h,"1,480,000",S,Biggin,3/12/2016,2.5,3067.0,1.0,1.0,202.0,0.0,0.0,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,"1,035,000",S,Biggin,4/02/2016,2.5,3067.0,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,"1,465,000",SP,Biggin,4/03/2017,2.5,3067.0,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019.0
5,Abbotsford,40 Federation La,3,h,"850,000",PI,Biggin,4/03/2017,2.5,3067.0,2.0,1.0,94.0,0.0,0.0,Yarra City Council,-37.7969,144.9969,Northern Metropolitan,4019.0
6,Abbotsford,55a Park St,4,h,"1,600,000",VB,Nelson,4/06/2016,2.5,3067.0,1.0,2.0,120.0,142.0,2014.0,Yarra City Council,-37.8072,144.9941,Northern Metropolitan,4019.0


In [41]:
house_data_cleaned['Postcode'] = house_data_cleaned['Postcode'].astype(int)
house_data_cleaned['Bathroom'] = house_data_cleaned['Bathroom'].astype(int)
house_data_cleaned['Car'] = house_data_cleaned['Car'].astype(int)
house_data_cleaned['Landsize'] = house_data_cleaned['Landsize'].astype(int)
house_data_cleaned['BuildingArea'] = house_data_cleaned['BuildingArea'].astype(int)
house_data_cleaned['YearBuilt'] = house_data_cleaned['YearBuilt'].astype(int)
house_data_cleaned['Propertycount'] = house_data_cleaned['Propertycount'].astype(int)

house_data_cleaned.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
1,Abbotsford,85 Turner St,2,h,"1,480,000",S,Biggin,3/12/2016,2.5,3067,1,1,202,0,0,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019
2,Abbotsford,25 Bloomburg St,2,h,"1,035,000",S,Biggin,4/02/2016,2.5,3067,1,0,156,79,1900,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019
4,Abbotsford,5 Charles St,3,h,"1,465,000",SP,Biggin,4/03/2017,2.5,3067,2,0,134,150,1900,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019
5,Abbotsford,40 Federation La,3,h,"850,000",PI,Biggin,4/03/2017,2.5,3067,2,1,94,0,0,Yarra City Council,-37.7969,144.9969,Northern Metropolitan,4019
6,Abbotsford,55a Park St,4,h,"1,600,000",VB,Nelson,4/06/2016,2.5,3067,1,2,120,142,2014,Yarra City Council,-37.8072,144.9941,Northern Metropolitan,4019


In [45]:
school_data_cleaned = school_data.drop(["SCHOOL_NO",
                                        "Address_Line_2",
                                        "Postal_Address_Line_1",
                                        "Postal_Address_Line_2",
                                        "Postal_Town",
                                        "Postal_State",
                                        "Postal_Postcode"], axis = "columns")

school_data_cleaned = school_data_cleaned.rename({"X": "Lng",
                                                  "Y": "Lat",
                                                  "Address_Line_1": "Address",
                                                  "Address_Town": "Suburb",
                                                  "Address_State": "State",
                                                  "Address_Postcode": "Postcode"}, axis = "columns")


school_data_cleaned.head()

,Education_Sector,Entity_Type,School_Name,School_Type,School_Status,Address,Suburb,State,Postcode,Full_Phone_No,LGA_ID,LGA_Name,Lng,Lat
0,Government,1,Alberton Primary School,Primary,O,21 Thomson Street,Alberton,VIC,3971,03 5183 2412,681,Wellington (S),146.666601,-38.617713
1,Government,1,Allansford and District Primary School,Primary,O,Frank Street,Allansford,VIC,3277,03 5565 1382,673,Warrnambool (C),142.590393,-38.386281
2,Government,1,Avoca Primary School,Primary,O,118 Barnett Street,Avoca,VIC,3467,03 5465 3176,599,Pyrenees (S),143.475649,-37.084502
3,Government,1,Avenel Primary School,Primary,O,40 Anderson Street,Avenel,VIC,3664,03 5796 2264,643,Strathbogie (S),145.234722,-36.901368
4,Government,1,Warrandyte Primary School,Primary,O,5-11 Forbes Street,Warrandyte,VIC,3113,03 9844 3537,421,Manningham (C),145.213980,-37.742675


In [43]:
school_data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2291 entries, 0 to 2290
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Education_Sector  2291 non-null   object 
 1   Entity_Type       2291 non-null   int64  
 2   School_Name       2291 non-null   object 
 3   School_Type       2291 non-null   object 
 4   School_Status     2291 non-null   object 
 5   Address_Line_1    2291 non-null   object 
 6   Address_Town      2291 non-null   object 
 7   Address_State     2291 non-null   object 
 8   Address_Postcode  2291 non-null   int64  
 9   Full_Phone_No     2291 non-null   object 
 10  LGA_ID            2291 non-null   int64  
 11  LGA_Name          2291 non-null   object 
 12  X                 2291 non-null   float64
 13  Y                 2291 non-null   float64
dtypes: float64(2), int64(3), object(9)
memory usage: 250.7+ KB


In [46]:
house_school_df = pd.merge(house_data_cleaned, school_data_cleaned, on = "Postcode")

In [ ]:
house_school_df.head()

,Suburb_x,Address_x,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,School_Type,School_Status,Address_y,Suburb_y,State,Full_Phone_No,LGA_ID,LGA_Name,Lng,Lat
0,Abbotsford,85 Turner St,2,h,"1,480,000",S,Biggin,3/12/2016,2.5,3067,...,Primary,O,Lithgow Street,Abbotsford,VIC,03 9428 5977,735,Yarra (C),144.998814,-37.808979
1,Abbotsford,85 Turner St,2,h,"1,480,000",S,Biggin,3/12/2016,2.5,3067,...,Pri/Sec,O,1 St Heliers Street,ABBOTSFORD,VIC,03 9419 9229,735,Yarra (C),145.005225,-37.803664
2,Abbotsford,25 Bloomburg St,2,h,"1,035,000",S,Biggin,4/02/2016,2.5,3067,...,Primary,O,Lithgow Street,Abbotsford,VIC,03 9428 5977,735,Yarra (C),144.998814,-37.808979
3,Abbotsford,25 Bloomburg St,2,h,"1,035,000",S,Biggin,4/02/2016,2.5,3067,...,Pri/Sec,O,1 St Heliers Street,ABBOTSFORD,VIC,03 9419 9229,735,Yarra (C),145.005225,-37.803664
4,Abbotsford,5 Charles St,3,h,"1,465,000",SP,Biggin,4/03/2017,2.5,3067,...,Primary,O,Lithgow Street,Abbotsford,VIC,03 9428 5977,735,Yarra (C),144.998814,-37.808979


In [47]:
house_school_df = house_school_df.rename({"Suburb_x": "House Suburb",
                                          "Address_x": "House Address",
                                          "Address_y": "School Address",
                                          "Suburb_y": " School Suburb"}, axis = "columns")


In [48]:
house_school_df.head()

,House Suburb,House Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,School_Type,School_Status,School Address,School Suburb,State,Full_Phone_No,LGA_ID,LGA_Name,Lng,Lat
0,Abbotsford,85 Turner St,2,h,"1,480,000",S,Biggin,3/12/2016,2.5,3067,...,Primary,O,Lithgow Street,Abbotsford,VIC,03 9428 5977,735,Yarra (C),144.998814,-37.808979
1,Abbotsford,85 Turner St,2,h,"1,480,000",S,Biggin,3/12/2016,2.5,3067,...,Pri/Sec,O,1 St Heliers Street,ABBOTSFORD,VIC,03 9419 9229,735,Yarra (C),145.005225,-37.803664
2,Abbotsford,25 Bloomburg St,2,h,"1,035,000",S,Biggin,4/02/2016,2.5,3067,...,Primary,O,Lithgow Street,Abbotsford,VIC,03 9428 5977,735,Yarra (C),144.998814,-37.808979
3,Abbotsford,25 Bloomburg St,2,h,"1,035,000",S,Biggin,4/02/2016,2.5,3067,...,Pri/Sec,O,1 St Heliers Street,ABBOTSFORD,VIC,03 9419 9229,735,Yarra (C),145.005225,-37.803664
4,Abbotsford,5 Charles St,3,h,"1,465,000",SP,Biggin,4/03/2017,2.5,3067,...,Primary,O,Lithgow Street,Abbotsford,VIC,03 9428 5977,735,Yarra (C),144.998814,-37.808979


In [52]:
house_school_df.to_csv(output_file, index = False, encoding = "utf-8")